In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
print(tf.__version__)

import matplotlib.pyplot as plt
%matplotlib inline

from subprocess import check_output

1.10.0


In [3]:
!python -V

Python 3.6.5


In [25]:
data_train_file = "data/fashion-mnist_train.csv"
data_test_file = "data/fashion-mnist_test.csv"

df_train = pd.read_csv(data_train_file)
df_test = pd.read_csv(data_test_file)

In [26]:
# pd.options.display.max_columns = None   # 显示所有列数据
df_train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Preprocessing the dataset

In [27]:
# Note this returns numpy arrays
def get_features_labels(df):
    # Select all columns but the first
    features = df.values[:, 1:]/255
    # The first column is the label. Conveniently called 'label'
    labels = df.values[:, 0]   # labels = df['label'].values
    return features, labels

In [28]:
train_features, train_labels = get_features_labels(df_train)
test_features, test_labels = get_features_labels(df_test)

## One-hot encoding

In [29]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


In [30]:
train_labels_1hot = tf.keras.utils.to_categorical(train_labels) # 转化为 One-hot 独热编码
test_labels_1hot = tf.keras.utils.to_categorical(test_labels)

In [31]:
train_labels_1hot.shape

(60000, 10)

## Training Parameters

In [32]:
BATCH_SIZE = 128
EPOCHS = 2

## Create a Keras Model

In [33]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

# Create a TensorFlow optimizer, rather than using the Keras version
# This is currently necessary when working in eager mode
# optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

# We will now compile and print out a summary of our model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 30)                23550     
_________________________________________________________________
dense_10 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_11 (Dense)             (None, 10)                210       
Total params: 24,380
Trainable params: 24,380
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.fit(train_features, train_labels_1hot, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/2
60000/60000 [==============================] - 2s 37us/step - loss: 0.6684 - acc: 0.7774
Epoch 2/2
60000/60000 [==============================] - 2s 32us/step - loss: 0.4552 - acc: 0.8392


In [35]:
test_loss, test_acc = model.evaluate(test_features, test_labels_1hot)
print('test_acc:', test_acc)

10000/10000 [==============================] - 0s 24us/step
test_acc: 0.8376


## Convert Keras model to TensorFlow estimator

In [36]:
tf_classifier = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpmi64ya_z', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FA8E43E6A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Train TensorFlow model

In [37]:
model.input_names

['dense_9_input']

In [39]:
input_name = model.input_names[0]
input_name

'dense_9_input'

In [43]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={input_name: train_features},
    y=train_labels_1hot,
    batch_size=BATCH_SIZE,
    num_epochs=EPOCHS,
    shuffle=True) 

In [40]:
# We again use the same name from the keras model.input_names
feature_columns = [tf.feature_column.numeric_column(input_name, shape=784)]

In [41]:
classifier = tf_classifier

In [44]:
classifier.train(input_fn=train_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpmi64ya_z\keras_model.ckpt
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\User\AppData\Local\Temp\tmpmi64ya_z\model.ckpt.
INFO:tensorflow:loss = 0.40617767, step = 1
INFO:tensorflow:global_step/sec: 248.595
INFO:tensorflow:loss = 0.615465, step = 101 (0.404 sec)
INFO:tensorflow:global_step/sec: 386.984
INFO:tensorflow:loss = 0.45019746, step = 201 (0.258 sec)
INFO:tensorflow:global_step/sec: 389.215
INFO:tensorflow:loss = 0.35627732, step = 301 (0.256 sec)
INFO:tensorflow:global_step/sec: 385.493
INFO:tensorflow:loss = 0.31565055, step = 401 (0.259 sec)
INFO:tensorflow:global_step/sec: 386.231
INFO:tensorflow:loss = 0.32959384, step = 501 (0.259 sec)
INFO:tensorflow:glo

In [46]:
evaluate_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={input_name: test_features},
    y=test_labels_1hot,
    num_epochs=1,
    shuffle=False)

In [47]:
classifier.evaluate(input_fn=evaluate_input_fn)["accuracy"]

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-03-03-16:03:37
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpmi64ya_z\model.ckpt-939
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-03-03-16:03:37
INFO:tensorflow:Saving dict for global step 939: accuracy = 0.85957277, global_step = 939, loss = 0.40143552
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 939: C:\Users\User\AppData\Local\Temp\tmpmi64ya_z\model.ckpt-939


0.85957277

## Model Export

In [60]:
model.save("models/28_export/28_keras_model.h5")

In [61]:
feature_spec = {
    input_name: tf.FixedLenFeature(shape=[784], dtype=np.float32)
}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel(export_dir_base="models\\28_export",
                                          serving_input_receiver_fn=serving_fn)
export_dir = export_dir.decode("utf8")

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpmi64ya_z\model.ckpt-939
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: models\28_export\temp-b'1551629456'\saved_model.pb


In [62]:
export_dir

'models\\28_export\\1551629456'

In [ ]:
# !ls {export_dir}

In [63]:
!tar -zcvf exported_model.tgz {export_dir}   # 打包模型文件

a models/28_export/1551629456
a models/28_export/1551629456/saved_model.pb
a models/28_export/1551629456/variables
a models/28_export/1551629456/variables/variables.data-00000-of-00001
a models/28_export/1551629456/variables/variables.index


In [ ]:
# Model is zipped and ready for download. To unzip, run
# tar -zxvf exported_model.tgz

源代码: [Fashion-MNIST with Keras model to TF estimator](https://www.kaggle.com/yufengg/fashion-mnist-with-keras-model-to-tf-estimator)